<a href="https://colab.research.google.com/github/SimonADDA/ML-Poker-Hand/blob/main/ProjetML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Poker Hand prediction project

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sys
import warnings
warnings.simplefilter("ignore")

!pip install imbalanced-learn
import imblearn

# import required modules
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# reading csv files
df =  pd.read_csv('poker.data',sep=',',header=None)
print(df)

# Datacleaning of our Dataset

In [ ]:
df.head()

In [ ]:
df=df.rename(columns={0: "S1",1: "C1",2: "S2",3: "C2",4: "S3",5: "C3",6: "S4",7: "C4",8: "S5",9: "C5",10: "target"})

In [ ]:
# create a list of the values we want to assign for each condition
values = ['H', 'S', 'D', 'C']

# create a new column and use np.select to assign values to it using our lists as arguments
#df['S1'] = np.select(conditions, values)
df['SC1']=df['S1']+10*df['C1']
df['SC2']=df['S2']+10*df['C2']
df['SC3']=df['S3']+10*df['C3']
df['SC4']=df['S4']+10*df['C4']
df['SC5']=df['S5']+10*df['C5']

df.drop(columns=['S1', 'C1','S2','C2','S3','C3','S4','C4','S5','C5'], inplace=True)
# del df['S1']
df.head()

In [ ]:
df.dtypes

## EDA

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
# Nb de valeurs uniques par colonnes
valforcols = df.nunique()
valforcols

In [ ]:
#See the duplicated
df=df.drop_duplicates()
df.shape

In [ ]:
sns.heatmap(df.isna())

In [ ]:
# Let's analyse our target and its distribution
print(df['target'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(df['target'], color='g');

In [ ]:
_fig = df.hist()

In [ ]:
df['target'].describe()
sns.distplot(df['target'])
#skewness and kurtosis
print("Skewness: %f" % df['target'].skew())
print("Kurtosis: %f" % df['target'].kurt())

## Label encoder

In [ ]:
# generate binary values using get_dummies
#df = pd.get_dummies(df, columns=['SC1','SC2','SC3','SC4','SC5'])
#df.drop(columns=['SC1_134','SC2_134','SC3_134','SC4_134','SC5_134'], inplace=True)

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#define total sample size desired
#N = 50000
#perform stratified random sampling
#df=df.groupby('target', group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df))))).sample(frac=1).reset_index(drop=True)

In [ ]:
#We have the same destribution as before 

print(df['target'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(df['target'], color='g');

### SMOTE

In [ ]:
X = df.loc[:, df.columns != 'target']  # we only take the first two features. We could
                      # avoid this ugly slicing by using a two-dim dataset
y = df.target

In [ ]:
print('Original dataset shape %s' % Counter(y))

ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
X_res = pd.DataFrame(X_res)
y_res = pd.DataFrame(y_res)
y_res.iloc[:, 0].value_counts()

In [ ]:
X_res

# Models

In [ ]:
"""
================================
Nearest Neighbors Classification
================================

Sample usage of Nearest Neighbors classification.
It will plot the decision boundaries for each class.
"""
print(__doc__)
n_neighbors = 5

# import the iris data to play with, keep only two input variables in order to make the plots at the end of the script
X = df.loc[:, df.columns != 'target']  # we only take the first two features. We could
                      # avoid this ugly slicing by using a two-dim dataset
y = df.target

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors) # je précise la méthode et ses hyperparamètres
clf.fit(X, y) # je lance l'apprentissage

# prevision
clf.predict(X)
clf.score(X,y) # predit + calcul le score = la précision

In [ ]:

X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=random.seed())
print(X_train.shape)
print(X_test.shape)

In [ ]:
n_neighbors=5
clf = neighbors.KNeighborsClassifier(n_neighbors) # je précise la méthode et ses hyperparamètres
clf.fit(X_train, Y_train) # je lance l'apprentissage
prev_test = clf.predict(X_test)
sc_train = clf.score(X_train,Y_train)
sc_test = clf.score(X_test,Y_test)
print(sc_train)
print(sc_test)

In [ ]:
# Compute the confusion matrix between kmean label and iris types
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(Y_test,prev_test)
# the confusion matrix is difficult to read as labels assigned by kmeans are arbitrary
print(cm)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, prev_test))

from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(Y_test, prev_test))

In [ ]:
#Convert dataframe type to Numpy type
X_train=X_train.to_numpy()
Y_train=Y_train.to_numpy()

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=3, shuffle=True) # partages de validation

from sklearn import neighbors
scores=[]
for k in range(1,5):  # les différentes valeurs de k à tester
    score=0
    clf=neighbors.KNeighborsClassifier(k)
    for learn,test in kf.split(X_train): # boucle sur différents partages de validation
        X_app=X_train[learn]
        Y_app=Y_train[learn]
        clf.fit(X_app,Y_app)
        X_val=X_train[test]
        Y_val=Y_train[test]
        score+=clf.score(X_val,Y_val)
    scores.append(score)
print(scores)
#plt(scores)
k_opt=scores.index(max(scores)) + 1  # valeur optimale de k
print(k_opt)

In [ ]:
# Pour voir l'effet du partage ...

from sklearn.model_selection import KFold
kf=KFold(n_splits=4, shuffle=True) # partages de validation
XXX = X_train[:1000,:]  # je prends uniquement les 12 premières lignes
#print(XXX)
for learn,test in kf.split(XXX): # boucle sur différents partages de validation
  print("Learn")
  print(learn)
  print("test")
  print(test)

In [ ]:
#Make the test with 3 Neighbors

n_neighbors=3
clf = neighbors.KNeighborsClassifier(n_neighbors) # je précise la méthode et ses hyperparamètres
clf.fit(X_train, Y_train) # je lance l'apprentissage
prev_test = clf.predict(X_test)
sc_train = clf.score(X_train,Y_train)
sc_test = clf.score(X_test,Y_test)
print(sc_train)
print(sc_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(Y_test,prev_test)
print(cm)

## Decision tree


In [ ]:
from sklearn import tree
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=random.seed())
clf=tree.DecisionTreeClassifier(max_leaf_nodes=100,criterion='entropy')
clf.fit(X_train,Y_train)
prev_test = clf.predict(X_test)
#print(prev)
scoretree = clf.score(X_test,Y_test)
# Compute the confusion matrix between kmean label and iris types

cm=confusion_matrix(Y_test,prev_test)
# the confusion matrix is difficult to read as labels assigned by kmeans are arbitrary
print(cm)
print(scoretree)

### Ensemble methods

In [ ]:
#DecisonTreeClassifier
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
    random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print(scores.mean())

#RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print(scores.mean())

#ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=20, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print(scores.mean())

## Bagging

In [ ]:
#We try to make abagging with our best model, KNeighborsClassifier, and try to have a better result
clf = BaggingClassifier(KNeighborsClassifier(),
                            n_estimators=10, random_state=0)
clf.fit(X_train, Y_train)
prev_test = clf.predict(X_test)
scorerdmbagging = clf.score(X_test,Y_test)
print(scorerdmbagging)